# Chapter 4 - Exercise 2: Temperature Effects and Thermal Damage

## Learning Objectives
- Understand thermal damage mechanisms during bioprinting
- Apply Arrhenius equation to predict cell death
- Analyze temperature-sensitive gelation kinetics
- Model heat generation in photopolymerization
- Design thermal management strategies
- Optimize temperature-controlled bioprinting processes

## Background

**Temperature** is a critical process variable in bioprinting. Too cold → cells metabolically inactive and bioinks too viscous. Too hot → protein denaturation and cell death.

### Key Temperature Effects:

1. **Thermal Cell Damage**
   - Proteins denature above ~42°C
   - Arrhenius kinetics govern damage accumulation
   - Both temperature AND exposure time matter

2. **Temperature-Sensitive Gelation**
   - **Thermogelling** (e.g., Pluronic, Chitosan): Gel when heated
   - **Thermoreversible** (e.g., Gelatin, Agarose): Gel when cooled
   - Enables room-temperature printing → body-temperature gelation

3. **Heat Generation**
   - **Photopolymerization**: Exothermic reaction
   - **Friction**: In extrusion systems
   - **Laser exposure**: In SLS or LAB

### Arrhenius Damage Model:

Cell damage accumulates according to:

$$\Omega(t) = \int_0^t A \cdot e^{-E_a/(RT(\tau))} d\tau$$

where:
- Ω = damage integral (dimensionless)
- A = frequency factor (1/s)
- E_a = activation energy (J/mol)
- R = gas constant (8.314 J/mol·K)
- T = absolute temperature (K)

**Cell viability** after thermal exposure:
$$\text{Viability}(\%) = 100 \cdot e^{-\Omega}$$

### Temperature-Sensitive Gelation:

**Upper Critical Solution Temperature (UCST)** materials:
- Gel when **cooled** below T_gel
- Example: Gelatin, Agarose

**Lower Critical Solution Temperature (LCST)** materials:
- Gel when **heated** above T_gel
- Example: Pluronic F-127, pNIPAAm, Chitosan

**Gelation time** as function of temperature:
$$t_{gel} = t_0 \cdot \exp\left(\frac{E_{gel}}{R}\left(\frac{1}{T} - \frac{1}{T_0}\right)\right)$$

### Photopolymerization Heating:

Energy absorbed during light-based printing:
$$\Delta T = \frac{\eta \cdot E \cdot \alpha}{\rho \cdot c_p}$$

where:
- η = conversion efficiency (~0.1-0.3)
- E = exposure dose (mJ/cm²)
- α = absorption coefficient (1/cm)
- ρ = density (g/cm³)
- c_p = specific heat capacity (J/g·K)

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint, quad
from scipy.optimize import fsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready to explore thermal effects in bioprinting!")

## Part 1: Arrhenius Thermal Damage Model

In [ ]:
def arrhenius_damage(temperature_C, exposure_time_s, cell_type='mammalian'):
    """
    Calculate thermal damage using Arrhenius equation.
    
    Parameters:
    -----------
    temperature_C : float or array
        Temperature in Celsius
    exposure_time_s : float
        Exposure duration in seconds
    cell_type : str
        'mammalian', 'bacteria', or 'yeast'
    
    Returns:
    --------
    viability : float or array
        Cell viability percentage (0-100%)
    """
    # Cell-specific Arrhenius parameters
    params = {
        'mammalian': {'A': 7.39e104, 'Ea': 6.68e5},  # J/mol
        'bacteria': {'A': 5.0e98, 'Ea': 6.3e5},
        'yeast': {'A': 3.1e106, 'Ea': 6.9e5}
    }
    
    A = params[cell_type]['A']
    Ea = params[cell_type]['Ea']
    R = 8.314  # J/(mol·K)
    
    T_K = temperature_C + 273.15  # Convert to Kelvin
    
    # Calculate damage integral (assuming constant temperature)
    omega = A * exposure_time_s * np.exp(-Ea / (R * T_K))
    
    # Calculate viability
    viability = 100 * np.exp(-omega)
    
    return viability

def calculate_critical_time(temperature_C, target_viability=80, cell_type='mammalian'):
    """
    Calculate maximum safe exposure time for target viability.
    
    Parameters:
    -----------
    temperature_C : float
        Temperature in Celsius
    target_viability : float
        Target viability percentage
    cell_type : str
        Cell type
    
    Returns:
    --------
    t_critical : float
        Maximum safe exposure time (seconds)
    """
    params = {
        'mammalian': {'A': 7.39e104, 'Ea': 6.68e5},
        'bacteria': {'A': 5.0e98, 'Ea': 6.3e5},
        'yeast': {'A': 3.1e106, 'Ea': 6.9e5}
    }
    
    A = params[cell_type]['A']
    Ea = params[cell_type]['Ea']
    R = 8.314
    
    T_K = temperature_C + 273.15
    
    # Solve for time: Ω = -ln(viability/100)
    omega_target = -np.log(target_viability / 100)
    
    t_critical = omega_target / (A * np.exp(-Ea / (R * T_K)))
    
    return t_critical

print("✓ Arrhenius damage functions defined!")
print("\nFunctions available:")
print("  • arrhenius_damage(T, t, cell_type)")
print("  • calculate_critical_time(T, target_viability, cell_type)")

## Part 2: Thermal Damage Visualization

In [ ]:
# Create temperature and time ranges
temperatures = np.linspace(25, 60, 100)  # °C
times = np.array([1, 10, 60, 300, 600])  # seconds (1s, 10s, 1min, 5min, 10min)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Viability vs Temperature for different exposure times
colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c', '#9b59b6']
for t, color in zip(times, colors):
    viability = arrhenius_damage(temperatures, t, 'mammalian')
    if t < 60:
        label = f'{t} s'
    else:
        label = f'{t//60} min'
    ax1.plot(temperatures, viability, linewidth=3, color=color, label=label)

# Mark critical thresholds
ax1.axhline(80, color='green', linestyle='--', linewidth=2, alpha=0.7,
           label='Target (80%)')
ax1.axvline(37, color='blue', linestyle=':', linewidth=2, alpha=0.7,
           label='Body temp')
ax1.axvline(42, color='orange', linestyle=':', linewidth=2, alpha=0.7,
           label='Hyperthermia')

ax1.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax1.set_title('Thermal Damage: Temperature vs Viability\n(Mammalian Cells)',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True, ncol=2)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 105)

# Plot 2: Safe exposure time vs Temperature
temps_range = np.linspace(37, 55, 50)
safe_times = [calculate_critical_time(T, 80, 'mammalian') for T in temps_range]

ax2.semilogy(temps_range, safe_times, linewidth=3, color='#e74c3c')
ax2.fill_between(temps_range, 0.1, safe_times, alpha=0.3, color='green',
                label='Safe zone (>80% viability)')

# Mark important temperatures
for T_mark, label, color in [(37, 'Body temp', 'blue'),
                              (42, 'Hyperthermia', 'orange'),
                              (50, 'Severe damage', 'red')]:
    t_safe = calculate_critical_time(T_mark, 80, 'mammalian')
    ax2.plot(T_mark, t_safe, 'o', markersize=12, color=color,
            markeredgecolor='black', markeredgewidth=2)
    ax2.annotate(f'{label}\n{t_safe:.1f} s',
                xy=(T_mark, t_safe), xytext=(5, 20),
                textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.6),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ax2.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Maximum Safe Exposure Time (s)', fontsize=12, fontweight='bold')
ax2.set_title('Safe Exposure Time\n(for 80% viability target)',
             fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')
ax2.set_ylim(0.1, 1e5)

# Plot 3: Cell type comparison
cell_types = ['mammalian', 'bacteria', 'yeast']
colors_cells = ['#e74c3c', '#3498db', '#2ecc71']
exposure_time = 60  # 1 minute

for cell_type, color in zip(cell_types, colors_cells):
    viability = arrhenius_damage(temperatures, exposure_time, cell_type)
    ax3.plot(temperatures, viability, linewidth=3, color=color,
            label=cell_type.capitalize())

ax3.axhline(80, color='green', linestyle='--', linewidth=2, alpha=0.7)
ax3.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax3.set_title(f'Cell Type Comparison\n(Exposure time: {exposure_time} s)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 105)

# Plot 4: Time-Temperature equivalence (iso-viability curves)
viability_targets = [99, 95, 90, 80, 50]
colors_iso = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c', '#8e44ad']

temps_iso = np.linspace(37, 60, 100)
for viab, color in zip(viability_targets, colors_iso):
    times_iso = [calculate_critical_time(T, viab, 'mammalian') for T in temps_iso]
    ax4.semilogy(temps_iso, times_iso, linewidth=3, color=color,
                label=f'{viab}% viability')

ax4.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Exposure Time (s)', fontsize=12, fontweight='bold')
ax4.set_title('Time-Temperature Iso-Viability Curves\n(Mammalian Cells)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("   • Damage is exponentially sensitive to temperature")
print("   • At 37°C: cells tolerate hours of exposure")
print("   • At 42°C: damage accumulates in minutes")
print("   • At 50°C: cells die within seconds")
print("   • Mammalian cells most sensitive, bacteria most resistant")
print("   • Time-temperature trade-off: lower T → longer safe time")

## Part 3: Temperature-Sensitive Gelation

In [ ]:
def gelation_time(temperature_C, material='Pluronic'):
    """
    Calculate gelation time for temperature-sensitive materials.
    
    Parameters:
    -----------
    temperature_C : float or array
        Temperature in Celsius
    material : str
        'Pluronic' (LCST), 'Gelatin' (UCST), 'Agarose' (UCST)
    
    Returns:
    --------
    t_gel : float or array
        Gelation time in seconds
    """
    # Material parameters
    materials = {
        'Pluronic': {
            'T_gel': 25,  # °C (LCST)
            'E_gel': 50000,  # J/mol
            't_0': 60,  # s at reference temp
            'type': 'LCST'
        },
        'Gelatin': {
            'T_gel': 30,  # °C (UCST)
            'E_gel': 45000,
            't_0': 120,
            'type': 'UCST'
        },
        'Agarose': {
            'T_gel': 35,  # °C (UCST)
            'E_gel': 55000,
            't_0': 30,
            'type': 'UCST'
        }
    }
    
    params = materials[material]
    T_gel = params['T_gel']
    E_gel = params['E_gel']
    t_0 = params['t_0']
    gel_type = params['type']
    R = 8.314
    
    T_K = temperature_C + 273.15
    T0_K = T_gel + 273.15
    
    # Arrhenius-type temperature dependence
    t_gel = t_0 * np.exp((E_gel / R) * (1/T_K - 1/T0_K))
    
    # Set to infinity if outside gelation range
    if gel_type == 'LCST':
        # Gels when heated above T_gel
        t_gel = np.where(temperature_C < T_gel - 5, np.inf, t_gel)
    else:  # UCST
        # Gels when cooled below T_gel
        t_gel = np.where(temperature_C > T_gel + 5, np.inf, t_gel)
    
    return t_gel

# Visualize gelation behavior
temps_gel = np.linspace(15, 45, 200)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Gelation time vs Temperature for different materials
materials_plot = ['Pluronic', 'Gelatin', 'Agarose']
colors_mat = ['#3498db', '#e74c3c', '#2ecc71']

for material, color in zip(materials_plot, colors_mat):
    t_gel = gelation_time(temps_gel, material)
    # Plot only finite values
    t_gel_plot = np.where(np.isfinite(t_gel), t_gel, np.nan)
    ax1.semilogy(temps_gel, t_gel_plot, linewidth=3, color=color,
                label=material)

# Mark body and room temperature
ax1.axvline(25, color='gray', linestyle='--', linewidth=2, alpha=0.5,
           label='Room temp')
ax1.axvline(37, color='red', linestyle='--', linewidth=2, alpha=0.5,
           label='Body temp')

ax1.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Gelation Time (s)', fontsize=12, fontweight='bold')
ax1.set_title('Temperature-Dependent Gelation Kinetics',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3, which='both')
ax1.set_ylim(1, 1e4)

# Plot 2: LCST vs UCST behavior schematic
temps_schematic = np.linspace(15, 45, 100)
viscosity_LCST = 1 / (1 + np.exp(-(temps_schematic - 25) / 2))  # Sigmoid
viscosity_UCST = 1 / (1 + np.exp((temps_schematic - 30) / 2))  # Inverted sigmoid

ax2.plot(temps_schematic, viscosity_LCST, linewidth=3, color='#3498db',
        label='LCST (Pluronic)')
ax2.plot(temps_schematic, viscosity_UCST, linewidth=3, color='#e74c3c',
        label='UCST (Gelatin)')

# Shade gelation regions
ax2.axvspan(25, 45, alpha=0.2, color='blue', label='LCST gel zone')
ax2.axvspan(15, 30, alpha=0.2, color='red', label='UCST gel zone')

ax2.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Relative Viscosity / Gel Strength', fontsize=12, fontweight='bold')
ax2.set_title('LCST vs UCST Behavior',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Printing strategy with Pluronic (LCST)
time_print = np.linspace(0, 300, 500)
temp_profile = 25 + 12 * (1 - np.exp(-time_print / 60))  # Heat from 25°C to 37°C

ax3.plot(time_print, temp_profile, linewidth=3, color='#e74c3c',
        label='Temperature')
ax3.axhline(25, color='blue', linestyle='--', linewidth=2, alpha=0.7,
           label='T_gel (Pluronic)')
ax3.axhline(37, color='green', linestyle='--', linewidth=2, alpha=0.7,
           label='Body temperature')

# Annotate printing phases
ax3.annotate('Print at\nroom temp\n(liquid)', xy=(30, 26), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
ax3.annotate('Warming\n(gelation)', xy=(100, 31), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
ax3.annotate('Body temp\n(stable gel)', xy=(240, 36), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

ax3.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax3.set_title('LCST Bioprinting Strategy\n(Pluronic F-127)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)

# Plot 4: Printing strategy with Gelatin (UCST)
temp_profile_gelatin = 37 - 12 * (1 - np.exp(-time_print / 60))  # Cool from 37°C to 25°C

ax4.plot(time_print, temp_profile_gelatin, linewidth=3, color='#3498db',
        label='Temperature')
ax4.axhline(30, color='red', linestyle='--', linewidth=2, alpha=0.7,
           label='T_gel (Gelatin)')
ax4.axhline(25, color='gray', linestyle='--', linewidth=2, alpha=0.7,
           label='Room temperature')

# Annotate printing phases
ax4.annotate('Print warm\n(liquid)', xy=(30, 36), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7))
ax4.annotate('Cooling\n(gelation)', xy=(100, 31), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
ax4.annotate('Room temp\n(stable gel)', xy=(240, 26), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

ax4.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax4.set_title('UCST Bioprinting Strategy\n(Gelatin)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Temperature-Sensitive Gelation Insights:")
print("   • LCST materials (Pluronic): Print cool → Warm to gel")
print("   • UCST materials (Gelatin): Print warm → Cool to gel")
print("   • Enables room-temperature printing with body-temp gelation")
print("   • Gelation time strongly temperature-dependent")
print("   • Must balance printing time vs gelation kinetics")

## Part 4: Photopolymerization Heating

### 🎯 STUDENT TASK 1: Calculate Temperature Rise During DLP Printing

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Light exposure parameters
exposure_dose = 100        # mJ/cm² - try: 50, 100, 200
intensity = 10             # mW/cm² - try: 5, 10, 20

# Material properties (GelMA with LAP photoinitiator)
conversion_efficiency = 0.2  # 20% of light energy → heat - try: 0.1, 0.2, 0.3
absorption_coeff = 0.5      # 1/cm - try: 0.3, 0.5, 1.0
density = 1.0               # g/cm³ (water-like)
specific_heat = 4.18        # J/(g·K) (water-like)

# Initial temperature
T_initial = 25              # °C - try: 20, 25, 30

# ==========================================

# Calculate exposure time
exposure_time = exposure_dose / intensity  # seconds

# Calculate temperature rise
# ΔT = (η × E × α) / (ρ × cp)
delta_T = (conversion_efficiency * exposure_dose * absorption_coeff) / \
          (density * specific_heat * 1000)  # Convert mJ to J

T_final = T_initial + delta_T

# Calculate cell viability at peak temperature
viability_peak = arrhenius_damage(T_final, exposure_time, 'mammalian')

# Create visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Temperature Profile During Exposure',
                   'Energy Distribution',
                   'Cell Viability Impact',
                   'Parameter Summary'),
    specs=[[{'type': 'scatter'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'table'}]]
)

# Plot 1: Temperature vs time
time_profile = np.linspace(0, exposure_time, 100)
# Assume exponential heating
temp_profile = T_initial + delta_T * (1 - np.exp(-time_profile / (exposure_time/5)))

fig.add_trace(
    go.Scatter(x=time_profile, y=temp_profile,
              mode='lines', name='Temperature',
              line=dict(color='red', width=3)),
    row=1, col=1
)

# Add threshold lines
fig.add_hline(y=37, line_dash="dash", line_color="blue",
             annotation_text="Body temp", row=1, col=1)
fig.add_hline(y=42, line_dash="dash", line_color="orange",
             annotation_text="Damage threshold", row=1, col=1)

fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)

# Plot 2: Energy distribution
energy_absorbed = exposure_dose * absorption_coeff
energy_to_heat = energy_absorbed * conversion_efficiency
energy_to_polymerization = energy_absorbed * (1 - conversion_efficiency)

fig.add_trace(
    go.Bar(x=['Absorbed\nEnergy', 'Heat\nGeneration', 'Polymerization\nReaction'],
          y=[energy_absorbed, energy_to_heat, energy_to_polymerization],
          marker_color=['blue', 'red', 'green'],
          text=[f'{energy_absorbed:.1f}', f'{energy_to_heat:.1f}',
                f'{energy_to_polymerization:.1f}'],
          textposition='outside'),
    row=1, col=2
)

fig.update_yaxes(title_text="Energy (mJ/cm²)", row=1, col=2)

# Plot 3: Cell viability over exposure
time_viab = np.linspace(0, exposure_time, 100)
viability_time = [arrhenius_damage(T_final, t, 'mammalian') for t in time_viab]

fig.add_trace(
    go.Scatter(x=time_viab, y=viability_time,
              mode='lines', name='Viability',
              line=dict(color='green', width=3),
              fill='tozeroy', fillcolor='rgba(0,255,0,0.2)'),
    row=2, col=1
)

fig.add_hline(y=80, line_dash="dash", line_color="red",
             annotation_text="Target (80%)", row=2, col=1)

fig.update_xaxes(title_text="Exposure Time (s)", row=2, col=1)
fig.update_yaxes(title_text="Cell Viability (%)", range=[0, 105], row=2, col=1)

# Plot 4: Summary table
fig.add_trace(
    go.Table(
        header=dict(values=['Parameter', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=11)),
        cells=dict(values=[
            ['Exposure Dose', 'Intensity', 'Exposure Time',
             'Initial Temp', 'Temperature Rise', 'Final Temp',
             'Cell Viability', 'Status'],
            [f'{exposure_dose} mJ/cm²', f'{intensity} mW/cm²',
             f'{exposure_time:.1f} s', f'{T_initial}°C',
             f'+{delta_T:.2f}°C', f'{T_final:.2f}°C',
             f'{viability_peak:.1f}%',
             '✅ Safe' if viability_peak >= 80 else '⚠️ Damage Risk']
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=10))),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False,
                 title_text="Photopolymerization Heating Analysis")
fig.show()

# Detailed output
print("="*70)
print("PHOTOPOLYMERIZATION HEATING ANALYSIS")
print("="*70)

print(f"\nLight Exposure:")
print(f"  Dose: {exposure_dose} mJ/cm²")
print(f"  Intensity: {intensity} mW/cm²")
print(f"  Exposure time: {exposure_time:.1f} s")

print(f"\nEnergy Budget:")
print(f"  Total absorbed: {energy_absorbed:.1f} mJ/cm²")
print(f"  Heat generated: {energy_to_heat:.1f} mJ/cm² ({conversion_efficiency*100:.0f}%)")
print(f"  Polymerization: {energy_to_polymerization:.1f} mJ/cm²")

print(f"\nTemperature Effects:")
print(f"  Initial temperature: {T_initial}°C")
print(f"  Temperature rise: +{delta_T:.2f}°C")
print(f"  Peak temperature: {T_final:.2f}°C")

print(f"\nCell Viability:")
print(f"  Viability after exposure: {viability_peak:.1f}%")

if viability_peak >= 85:
    print(f"  ✅ STATUS: EXCELLENT - Very safe for cells")
elif viability_peak >= 80:
    print(f"  ✓ STATUS: GOOD - Meets viability target")
elif viability_peak >= 70:
    print(f"  ⚠️  STATUS: MARGINAL - Approaching damage threshold")
else:
    print(f"  ❌ STATUS: POOR - Significant cell damage expected")

print(f"\nRecommendations:")
if delta_T > 5:
    print(f"  • Temperature rise significant (>{delta_T:.1f}°C)")
    print(f"  • Consider: Lower intensity, shorter exposure, or cooling")
if T_final > 42:
    print(f"  • Peak temperature exceeds hyperthermia threshold")
    print(f"  • Risk of thermal protein denaturation")
if viability_peak < 80:
    print(f"  • Viability below target")
    print(f"  • Reduce exposure or use active cooling")

print("\n" + "="*70)

## Part 5: Thermal Management Strategies

In [ ]:
# Compare different thermal management approaches

strategies = {
    'No cooling': {
        'cooling_rate': 0,
        'T_ambient': 25,
        'color': '#e74c3c'
    },
    'Passive cooling': {
        'cooling_rate': 0.5,  # °C/s
        'T_ambient': 20,
        'color': '#f39c12'
    },
    'Active cooling': {
        'cooling_rate': 2.0,
        'T_ambient': 15,
        'color': '#3498db'
    },
    'Cryogenic cooling': {
        'cooling_rate': 10.0,
        'T_ambient': 4,
        'color': '#9b59b6'
    }
}

# Simulate temperature profile with heating and cooling
time_sim = np.linspace(0, 120, 500)  # 2 minutes
exposure_duration = 10  # seconds
heating_rate = 2  # °C/s during exposure

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Temperature profiles with different cooling
for strategy_name, params in strategies.items():
    T_profile = np.zeros_like(time_sim)
    T_profile[0] = 25  # Start at room temp
    
    for i in range(1, len(time_sim)):
        dt = time_sim[i] - time_sim[i-1]
        
        if time_sim[i] <= exposure_duration:
            # Heating during exposure
            T_profile[i] = T_profile[i-1] + heating_rate * dt
        else:
            # Cooling after exposure
            cooling = params['cooling_rate'] * dt
            T_profile[i] = max(T_profile[i-1] - cooling, params['T_ambient'])
    
    ax1.plot(time_sim, T_profile, linewidth=3, color=params['color'],
            label=strategy_name)

# Mark critical temperatures
ax1.axhline(37, color='blue', linestyle='--', linewidth=2, alpha=0.5,
           label='Body temp')
ax1.axhline(42, color='red', linestyle='--', linewidth=2, alpha=0.5,
           label='Damage threshold')
ax1.axvline(exposure_duration, color='gray', linestyle=':', linewidth=2,
           alpha=0.5, label='End of exposure')

ax1.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax1.set_title('Thermal Management Strategies',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative thermal damage
for strategy_name, params in strategies.items():
    T_profile = np.zeros_like(time_sim)
    T_profile[0] = 25
    damage = np.zeros_like(time_sim)
    
    for i in range(1, len(time_sim)):
        dt = time_sim[i] - time_sim[i-1]
        
        if time_sim[i] <= exposure_duration:
            T_profile[i] = T_profile[i-1] + heating_rate * dt
        else:
            cooling = params['cooling_rate'] * dt
            T_profile[i] = max(T_profile[i-1] - cooling, params['T_ambient'])
        
        # Calculate damage accumulation (simplified Arrhenius)
        if T_profile[i] > 37:
            A = 7.39e104
            Ea = 6.68e5
            R = 8.314
            T_K = T_profile[i] + 273.15
            damage[i] = damage[i-1] + A * dt * np.exp(-Ea / (R * T_K))
        else:
            damage[i] = damage[i-1]
    
    viability = 100 * np.exp(-damage)
    ax2.plot(time_sim, viability, linewidth=3, color=params['color'],
            label=strategy_name)

ax2.axhline(80, color='green', linestyle='--', linewidth=2, alpha=0.7,
           label='Target (80%)')
ax2.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax2.set_title('Viability with Different Cooling Strategies',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 105)

# Plot 3: Cooling effectiveness comparison
strategy_names = list(strategies.keys())
cooling_rates = [strategies[s]['cooling_rate'] for s in strategy_names]
colors_strat = [strategies[s]['color'] for s in strategy_names]

bars = ax3.barh(strategy_names, cooling_rates, color=colors_strat,
               alpha=0.7, edgecolor='black', linewidth=2)

for bar, value in zip(bars, cooling_rates):
    ax3.text(value + 0.3, bar.get_y() + bar.get_height()/2,
            f'{value:.1f} °C/s',
            va='center', fontweight='bold', fontsize=11)

ax3.set_xlabel('Cooling Rate (°C/s)', fontsize=12, fontweight='bold')
ax3.set_title('Cooling Effectiveness Comparison',
             fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# Plot 4: Final viability comparison
final_viabilities = []
for strategy_name in strategy_names:
    params = strategies[strategy_name]
    # Simplified calculation for final viability
    peak_temp = 25 + heating_rate * exposure_duration
    
    # Estimate average temp during cooling
    if params['cooling_rate'] > 0:
        cooling_time = (peak_temp - params['T_ambient']) / params['cooling_rate']
        avg_temp = (peak_temp + params['T_ambient']) / 2
    else:
        cooling_time = 110
        avg_temp = peak_temp
    
    # Total thermal exposure
    viab_exposure = arrhenius_damage(peak_temp, exposure_duration, 'mammalian')
    viab_cooling = arrhenius_damage(avg_temp, min(cooling_time, 110), 'mammalian')
    
    # Combined effect (simplified)
    final_viab = min(viab_exposure, viab_cooling)
    final_viabilities.append(final_viab)

bars2 = ax4.bar(strategy_names, final_viabilities, color=colors_strat,
               alpha=0.7, edgecolor='black', linewidth=2)

for bar, value in zip(bars2, final_viabilities):
    ax4.text(bar.get_x() + bar.get_width()/2, value + 2,
            f'{value:.1f}%',
            ha='center', fontweight='bold', fontsize=11)

ax4.axhline(80, color='green', linestyle='--', linewidth=2, alpha=0.7,
           label='Target')
ax4.set_ylabel('Final Cell Viability (%)', fontsize=12, fontweight='bold')
ax4.set_title('Final Viability Comparison',
             fontsize=13, fontweight='bold')
ax4.set_ylim(0, 105)
ax4.legend(loc='lower right')
ax4.grid(True, alpha=0.3, axis='y')
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=15, ha='right')

plt.tight_layout()
plt.show()

print("\n💡 Thermal Management Insights:")
print("   • Active cooling dramatically improves viability")
print("   • Faster cooling → less accumulated thermal damage")
print("   • Even passive cooling helps significantly")
print("   • Cryogenic cooling best but adds complexity")
print("   • Trade-off: cooling effectiveness vs. system complexity")

## Part 6: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 2: KEY LEARNING POINTS")
print("="*80)
print("""
1. ARRHENIUS THERMAL DAMAGE
   → Damage = ∫ A·exp(-Ea/RT) dt
   → Exponentially sensitive to temperature
   → Time-temperature equivalence
   → At 37°C: hours safe; at 50°C: seconds fatal

2. CRITICAL TEMPERATURE THRESHOLDS
   → 37°C: Normal body temperature (safe indefinitely)
   → 42°C: Hyperthermia threshold (damage in minutes)
   → 45°C: Rapid protein denaturation
   → 50°C+: Cell death in seconds

3. TEMPERATURE-SENSITIVE GELATION
   → LCST materials (Pluronic): Gel when HEATED
     • Print cool (liquid) → warm to gel
     • Ideal for room temp printing → body temp gelation
   → UCST materials (Gelatin): Gel when COOLED
     • Print warm (liquid) → cool to gel
     • Requires temperature control

4. PHOTOPOLYMERIZATION HEATING
   → Light absorption → heat generation
   → ΔT = (η×E×α)/(ρ×cp)
   → Typical rise: 1-5°C
   → Higher intensity → more heating
   → Conversion efficiency critical parameter

5. THERMAL MANAGEMENT IS ESSENTIAL
   → Active cooling dramatically improves outcomes
   → Even passive cooling helps
   → Faster cooling → less damage accumulation
   → Balance: effectiveness vs complexity

6. CELL TYPE SENSITIVITY VARIES
   → Mammalian cells: Most sensitive
   → Bacteria: More resistant
   → Yeast: Intermediate
   → Choose parameters based on cell type

7. DESIGN STRATEGIES
   → For photopolymerization:
     • Use lower intensity, longer exposure
     • Optimize photoinitiator concentration
     • Implement cooling system
   → For thermogelling:
     • Match material to process temperature
     • Control gelation kinetics
     • Minimize time at damaging temperatures

8. PRACTICAL LIMITS
   → Maximum safe printing time at 37°C: essentially unlimited
   → At 42°C: ~5-10 minutes
   → At 45°C: ~1-2 minutes
   → At 50°C: ~10-30 seconds
   → Design processes to stay in safe zones

9. MONITORING AND CONTROL
   → Real-time temperature sensing essential
   → Feedback control for heating/cooling
   → Infrared thermography useful
   → Consider heat capacity of entire system

10. INTEGRATION WITH OTHER FACTORS
    → Temperature affects viscosity (rheology)
    → Impacts mass transport (diffusion rates)
    → Influences gelation kinetics
    → Must optimize holistically
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**Calculate the safe exposure time for mammalian cells at 45°C to maintain >80% viability. How does this compare to the safe time at 40°C?**

### Question 2
**Explain why Pluronic F-127 is useful for bioprinting. What temperature strategy would you use and why?**

### Question 3
**A DLP printer uses 20 mW/cm² intensity with 150 mJ/cm² total dose. If 25% of absorbed light energy becomes heat, estimate the temperature rise in a water-based bioink.**

### Question 4
**Design a thermal management system for a photopolymerization bioprinter. Consider cooling methods, temperature sensing, and control strategies.**

### Question 5
**Compare LCST and UCST materials for cardiac tissue bioprinting. Which would you choose and why? Consider printing temperature, gelation kinetics, and cell viability.**

## 🎯 Congratulations!

You've completed Exercise 2: Temperature Effects and Thermal Damage!

**You now understand:**
- ✓ Arrhenius thermal damage kinetics
- ✓ Critical temperature thresholds for cells
- ✓ Temperature-sensitive gelation (LCST vs UCST)
- ✓ Photopolymerization heating
- ✓ Thermal management strategies
- ✓ Time-temperature equivalence

**Next Steps:**
- Continue to Exercise 3: Mass Transport and Diffusion
- Explore how temperature affects other bioprinting parameters
- Design temperature-controlled bioprinting protocols

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*